# A Brief Introduction to Bayesian Data Analysis

<i>Version 2</i>

<b><i>Probability Theory</i></b> is a subfield of Mathematics that is concerned with mathematical models of random phenomena.


<i><b>Mathematical Statistics</b></i> is a subfield of Probability Theory concerned with the derivation of features of a probability model based on a set of <b>Random Variables (RV)</b> from the model.

<i><b>Applied Statistics</b></i> utilizes the results of Mathematical Statistics by replacing the set of RVs with actual observed data to make <b>statistical inferences</b>.

How one conducts and interprets statistical inferences has been the subject of much debate for many decades.  In the 20th century, the debate was mostly over the <b>Frequentist</b> vs. the <b>Fisherian</b> approaches.  The <b>Bayesian</b> approach was also around during that time, but did not become practical until the 1990s with the advent of efficient simulation methods, such as <b>Markov Chain Monte Carlo (MCMC)</b>, and widespread access to powerful computers. Here, I'll refer the Frequentist and Fisherian approaches as the <b>Classical</b> approach. Today, the debate over how to conduct and interpret statistical inference is more about the Classical vs. Bayesian approaches.

The purpose of this notebook is to provide a brief and (hopefully) hype-free look at <b>Bayesian Data Analysis (BDA)</b>.

To begin, statistical analyses can be viewed in terms of the mathematical assumptions they make. Three broad categories are listed below, where each one makes successively greater assumptions than the previous one:
1. Exploratory -- No probability model; just descriptive statistics
1. Classical -- Probability model with fixed parameter
1. Bayesian -- Probability model with random parameter (with its own model & fixed <i>hyper-parameter</i>)

We'll go into a bit more depth on each of these three categories in the following sections.

### 1. Exploratory Data Analysis

No probability model.

#### Examples

* Average
* Median
* Quantiles
* Range
* Minimum/Maximum
* Histogram 

### 2. Classical Data Analysis

Assumes a random variable, $X$, with a specific probability distribution, $F$ and a fixed parameter $\theta$:

$X \sim F(x;\theta)$

#### Example

$\Phi(x) = \frac{1}{{\sqrt {2\pi}}}e^{-x^2/2}$ is the <b>standard normal distribution</b> function.

Let $\theta = \begin{pmatrix} \mu \\ \sigma \end{pmatrix}$, where $- \infty < \mu < + \infty$ and $\sigma > 0$,

then $F(x;\theta) \equiv \Phi({{x - \mu} \over \sigma})$ is one of the most commonly used probabilistic models in all of statistics.

### 3. Bayesian Data Analysis

<i>(NOTE: This starts out like the Classical approach, except that it goes a step further by assuming that the parameter itself has a model.)</i>

Assumes a random variable, $X$, with a specific probability distribution, $F$, and <b>random</b> parameter, $\Theta$, with its own distribution, $G$, and fixed parameter, $\gamma$, sometimes called a hyper-parameter:

$X \sim F(x;\Theta)$ where $\Theta \sim G(\theta;\gamma)$

$G$ is called a <b><i>prior</i></b> distribution.